# Anime Generation
## DCGAN 

> Data pre-processing [here](https://www.tensorflow.org/tutorials/load_data/images)

In [ ]:
from IPython.display import Audio
from IPython import display
the_voice = './sounds/chose_a_voice.wav'
the_creation = './sounds/You_created_me.wav'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# Error: Node: 'sequential_3/dropout_4/dropout/random_uniform/RandomUniform'
# OOM when allocating tensor with shape[256,180,120,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
# 	 [[{{node sequential_3/dropout_4/dropout/random_uniform/RandomUniform}}]]
# the solution to thi is report_tensor_allocations_upon_oom = True

import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

run_opts = tf.compat.v1.RunOptions(report_tensor_allocations_upon_oom=True)
runmeta = tf.compat.v1.RunMetadata()
tf.distribute.experimental.MultiWorkerMirroredStrategy(
    communication=tf.distribute.experimental.CollectiveCommunication.AUTO,
    cluster_resolver=None
)


import glob
import matplotlib.pyplot as plt
import numpy as np
import imageio
import tensorflow_docs
import PIL
import time
import wandb
from train.dcgan import DCGAN
# pip install wandb
# wandb login
# 65bacd21c40b0085e299e05ea94b552d5119b7bc

wandb.init(project="Anime Creation", entity="skycladai")

print("Note! You will need tensorflow-gpu version 2.8.0 or higher. Your version is", tf.__version__)
print(tf.config.list_physical_devices('GPU'))

Audio(url=the_voice, autoplay=True, rate=48000)

> Configuration

In [ ]:
# BATCH_SIZE = 254 # or 64 this cause memory allocation failure
BATCH_SIZE = 8
IMAGE_WIDTH = 240
IMAGE_LENGTH = 360
ORIGINAL_IMAGE_SIZE = (360, 240)
RGB = 3
BUFFER_SIZE = 58085
EPOCHS = 6
noise_dim = 100
num_examples_to_generate = 9
wandb.config = {
  "learning_rate": 0.001,
  "epochs": EPOCHS,
  "batch_size": BATCH_SIZE
}
DCGAN(batch_size= BATCH_SIZE, noise_dim= noise_dim, num_examples_to_generate=num_examples_to_generate)


# Load Dataset

In [ ]:
%%time

x_train = tf.keras.preprocessing.image_dataset_from_directory( directory="./dataset/", validation_split=0.3, subset="training" ,label_mode=None, batch_size=BATCH_SIZE, image_size=ORIGINAL_IMAGE_SIZE, seed=123 , shuffle=BUFFER_SIZE)
x_test = tf.keras.preprocessing.image_dataset_from_directory( directory="./dataset/", validation_split=0.3, subset="validation" ,label_mode=None, batch_size=BATCH_SIZE, image_size=ORIGINAL_IMAGE_SIZE, seed=123, shuffle=BUFFER_SIZE)
    
plt.figure(figsize=(10, 10))
plt.title("Training Images")
for images in x_train.take(1):
  for i in range(6):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.axis("off")


In [ ]:
for image_batch in x_train:
    print(image_batch.shape)
    break

In [ ]:
%%time
normalization_layer = tf.keras.layers.Rescaling(1./255)
# Batch and shuffle the data
def configure_for_performance(ds, name):
    ds = ds.cache(name)
    ds = ds.shuffle(buffer_size=2000)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

normalized_ds = x_train.map(lambda x: (normalization_layer(x)))
image_batch = next(iter(normalized_ds))
# Cache keeps the images in memory after they're loaded off disk during the first epoch. 
# This will ensure the dataset does not become a bottleneck while training your model. 
# Prefetch overlaps data preprocessing and model execution while training.
# This allows later elements to be prepared while the current element is being processed. 
# This often improves latency and throughput, at the cost of using additional memory to store prefetched elements.
AUTOTUNE = tf.data.AUTOTUNE
print(normalized_ds)

train_ds = configure_for_performance(normalized_ds, "./cache/training_cashe")
val_ds = configure_for_performance(x_test, "./cache/testing_cashe")
for image_batch in train_ds:
    print(image_batch.shape)
    break

print("In range between: ", np.min(image_batch[0])," and: ", np.max(image_batch[0]))

## DCGAN 
> Generator

In [ ]:
%%time

noise = tf.random.normal([1, 100])
generated_image = DCGAN.generator(noise, training=False)
print(generated_image.shape)
plt.imshow(generated_image[0, :, :, 0])
DCGAN.generator.summary()

> Discriminator 

In [ ]:
decision = DCGAN.discriminator(generated_image)
print(decision)
DCGAN.discriminator.summary()

> Save Checkpoints

## Training DCGAN

> Training definitions 

> Actual Train

In [ ]:
%%time

DCGAN.train(train_ds, EPOCHS)

Audio(url=the_creation, autoplay=True, rate=48000)

In [ ]:
DCGAN.checkpoint.restore(tf.train.latest_checkpoint(DCGAN.checkpoint_dir))

 > Create a GIF

In [ ]:
%%time
# Display a single image using the epoch number
def display_image(epoch_no):
  return PIL.Image.open('./generated_images/image_at_epoch_{:04d}.png'.format(epoch_no))
display_image(6)


In [ ]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

In [ ]:
import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)